# Using NER to Tag Data

We are using the dataset pulled from */r/investing* and the SpaCy NER code covered in the previous sections.

In [1]:
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_sm')

2023-05-06 21:40:18.026568: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-06 21:40:18.050734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-06 21:40:18.050909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


The NER code will be reformated into a function that will consume a length of text, and return a list of mentioned organizations:

In [2]:
def get_orgs(text):
    # process the text with our SpaCy model to get named entities
    doc = nlp(text)
    # initialize list to store identified organizations
    org_list = []
    # loop through the identified entities and append ORG entities to org_list
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

## Applying NER

All we need to do now is load in the */r/investing* data and apply the `get_orgs` function to our text column to create a new `organizations` column.

1. Load the data and view the top five rows with `df.head()`:

In [5]:
df = pd.read_csv('./data/reddit_investing.csv', sep='|')
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,t3_139yfid,1683398660,investing,Low-end primary residence vs index stock portf...,"This is quite anecdotal, and only applies to s...",1.00,3,0,3
1,t3_139y0se,1683397773,investing,Loaning shares to short sellers?,So my understanding about short-selling is tha...,0.67,1,0,1
2,t3_139xtc6,1683397318,investing,AI doesn’t beat human intelligence… Charlie Mu...,Asked ChatGBT what it thinks about the annual ...,0.32,0,0,0
3,t3_139wtm2,1683395147,investing,Taxes on dividend income vs salary?,I'm 31 and I make $115k USD pretax and my inve...,0.76,12,0,12
4,t3_139wt3y,1683395120,investing,Would the demystification and increase in publ...,There are a couple of factors I'm thinking of:...,0.84,26,0,26


2. Extract mentioned organizations from `selftext` and add to a new column called `organizations`:

In [6]:
df['organizations'] = df['selftext'].apply(get_orgs)
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations
0,t3_139yfid,1683398660,investing,Low-end primary residence vs index stock portf...,"This is quite anecdotal, and only applies to s...",1.00,3,0,3,[150k]
1,t3_139y0se,1683397773,investing,Loaning shares to short sellers?,So my understanding about short-selling is tha...,0.67,1,0,1,[]
2,t3_139xtc6,1683397318,investing,AI doesn’t beat human intelligence… Charlie Mu...,Asked ChatGBT what it thinks about the annual ...,0.32,0,0,0,"[YouTube, AI]"
3,t3_139wtm2,1683395147,investing,Taxes on dividend income vs salary?,I'm 31 and I make $115k USD pretax and my inve...,0.76,12,0,12,[]
4,t3_139wt3y,1683395120,investing,Would the demystification and increase in publ...,There are a couple of factors I'm thinking of:...,0.84,26,0,26,[]


*(This step can take a long time to run. It can be useful to break larger datasets into more manageable chunks if required)*

Now we have a list of all mentioned organizations contained within the `organizations` column. We will now take the full column, merge each list, and use `Counter` to create a frequency table of organization mentions.

In [7]:
# merge organizations column into one big list
orgs = df['organizations'].to_list()
orgs = [org for sublist in orgs for org in sublist]
orgs[:10]

['150k',
 'YouTube',
 'AI',
 'WeBull',
 'Interactive Brokers',
 'Here](https://www.bogleheads.org/wiki/Nonresident_alien_taxation#Is_interest_taxable_for_a_nonresident_alien',
 'IRS',
 'Wiki',
 'FAQ',
 'Petrodollar']

In [12]:
from collections import Counter

In [13]:
# create dictionary of organization mention frequency
org_freq = Counter(orgs)

We now have a *Counter* dictionary containing all of our organization labels as *keys*, and their mention frequency as *values*. The `most_common(n)` method allows us to view the **n** most frequently mentioned organizations:

In [14]:
org_freq.most_common(10)

[('Wiki', 47),
 ('FAQ', 47),
 ('Fed', 35),
 ('FDIC', 27),
 ('Fidelity', 23),
 ('VOO', 23),
 ('Vanguard', 20),
 ('USD', 18),
 ('AI', 14),
 ('fed', 13)]

Clearly there is a need to do some further pruning of the data to remove non-organization labels like *EV* (electric vehicle), *COVID*, *ETF* (exchange trading fund), *NYSE*, and *SPAC* (special acquisition company). Depending on the use-case it may even be useful to keep a few of these, or remove a few others like *SEC* and *FDA*.

To do this, we would create a custom list and implement it in our `get_orgs` function like so:

In [15]:
BLACKLIST = ['ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda']

def get_orgs(text):
    doc = nlp(text)
    org_list = []
    for entity in doc.ents:
        # here we modify the original code to check that entity text is not equal to one of our 'blacklisted' organizations
        # (we also add .lower() to lowercase the text, this allows us to match both 'nyse' and 'NYSE' with just 'nyse')
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

If we then apply this new `get_org` function to our dataframe we will see that *ETF* that originally appeared alongside *ARK* in the second row is no longer included:

In [16]:
df['organizations'] = df['selftext'].apply(get_orgs)
df.head()

,id,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations
0,t3_139yfid,1683398660,investing,Low-end primary residence vs index stock portf...,"This is quite anecdotal, and only applies to s...",1.00,3,0,3,[150k]
1,t3_139y0se,1683397773,investing,Loaning shares to short sellers?,So my understanding about short-selling is tha...,0.67,1,0,1,[]
2,t3_139xtc6,1683397318,investing,AI doesn’t beat human intelligence… Charlie Mu...,Asked ChatGBT what it thinks about the annual ...,0.32,0,0,0,"[YouTube, AI]"
3,t3_139wtm2,1683395147,investing,Taxes on dividend income vs salary?,I'm 31 and I make $115k USD pretax and my inve...,0.76,12,0,12,[]
4,t3_139wt3y,1683395120,investing,Would the demystification and increase in publ...,There are a couple of factors I'm thinking of:...,0.84,26,0,26,[]


We can save our dataframe with the new *organizations* column to file for later use.

In [17]:
df.to_csv('./data/processed/reddit_investing_ner.csv', sep='|', index=False)